In [1]:
import geopandas as gpd, pandas as pd

In [2]:

# Path corretti dei file shapefile
files = [
    "Aree protette/aree_riequilibrio_ecologico_rer201812/aree_riequilibrio_ecologico_rer201812.shp",
    "Aree protette/paesaggi_protetti_rer202007/paesaggi_protetti_rer202007.shp",
    "Aree protette/parchi_riserve_rer202109/parchi_riserve_rer202109.shp",
    "Aree protette/parchi_riserve_rer202109/zone_parchi_riserve_rer202109.shp",
    "Aree protette/siczps_rer202111/siczps_rer202111.shp"
]

# Caricamento dei GeoDataFrames e correzione delle geometrie non valide
gdfs = []
for file in files:
    gdf = gpd.read_file(file)
    gdf['geometry'] = gdf.geometry.buffer(0)  # Correzione delle geometrie
    gdfs.append(gdf)

# Concatenazione dei GeoDataFrames in uno solo
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Dissoluzione di tutte le geometrie in un'unica geometria
union_gdf = combined_gdf.dissolve()

# Caricamento dei confini della provincia di Bologna
bologna_borders = gpd.read_file("Limiti01012023/ProvCM01012023/ProvCM01012023_WGS84.shp")
bologna_borders = bologna_borders[bologna_borders['COD_PROV'] == 37].reset_index(drop=True)

# Assicurarsi che i CRS corrispondano
if union_gdf.crs != bologna_borders.crs:
    # Riconvertire il CRS di union_gdf per corrispondere a quello di bologna_borders
    union_gdf = union_gdf.to_crs(bologna_borders.crs)

# Taglio del GeoDataFrame unificato con i confini di Bologna
final_gdf = gpd.clip(union_gdf, bologna_borders)

In [5]:
# Salva il risultato finale
final_gdf.to_file("restricted_analysis/bologna_protected_areas.shp")